In [1]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
import matplotlib.pyplot as plt
import torch

c:\Users\qtly_u\.conda\envs\tf39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
image = Image.open("000000039769.jpg")
plt.imshow(image)
plt.show()

: 

: 

# 1. pretrained model & processor load

In [3]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# 2. preprocessing

In [4]:
candidates = ["three cats lying on the couch", "a photo of a cat", "a photo of a dog", "a lion", "two cats lying on the cushion"]

# 프로세서(ClIPPprocessor)에 텍스트 및 이미지를 입력하여 인코딩
inputs = processor(text=candidates, images=image, return_tensors="pt", padding=True)

In [5]:
print(inputs.keys())
print(inputs)

dict_keys(['input_ids', 'attention_mask', 'pixel_values'])
{'input_ids': tensor([[49406,  2097,  3989,  7175,   525,   518, 12724, 49407],
        [49406,   320,  1125,   539,   320,  2368, 49407, 49407],
        [49406,   320,  1125,   539,   320,  1929, 49407, 49407],
        [49406,   320,  5567, 49407, 49407, 49407, 49407, 49407],
        [49406,  1237,  3989,  7175,   525,   518, 20853, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]]), 'pixel_values': tensor([[[[ 0.5873,  0.5873,  0.6165,  ...,  0.0617,  0.0471, -0.0259],
          [ 0.5727,  0.5727,  0.6603,  ...,  0.1201,  0.0763,  0.0909],
          [ 0.5873,  0.5435,  0.6165,  ...,  0.0325,  0.1201,  0.0617],
          ...,
          [ 1.8719,  1.8573,  1.8719,  ...,  1.3902,  1.4340,  1.4194],
          [ 1.8281,  1.8719,  1.8427,  ...,  1.4486,  1.4340,  1.5070],
          [ 1.

In [7]:
print(inputs['pixel_values'].shape)

torch.Size([1, 3, 224, 224])


[1, 3, 224, 224] : index, colormap, resolustion nxn

In [8]:
# 인코딩된 텍스트
print(inputs['input_ids'][0])
# 위 결과를 디코딩한 텍스트 확인
print(processor.tokenizer.decode(inputs['input_ids'][0]))

tensor([49406,  2097,  3989,  7175,   525,   518, 12724, 49407])
<|startoftext|>three cats lying on the couch <|endoftext|>


# 3. Inference

In [10]:
model.eval()
outputs = model(**inputs)
# **inputs에서의 ** 표시는 inputs 변수가 키(key)와 값(value)로 이루어져 있을 때
# input 변수에 담긴 키와 값을 모두 모델에 입력하는 용도임

print(outputs.keys())

odict_keys(['logits_per_image', 'logits_per_text', 'text_embeds', 'image_embeds', 'text_model_output', 'vision_model_output'])


- 'text_embeds', 'image_embeds' : 텍스트와 이미지 은닉층의 벡터표현
- 'text_model_output', 'vision_model_output': 텍스트와 이미지 출력층의 벡터표현

In [13]:
# 로짓은 딥러닝 모델의 최종층에서 생성된 중간출력값이며 일종의 확률 -> 활성화함수에 입력하는 값
logits_per_image = outputs.logits_per_image
print(logits_per_image)

tensor([[28.7515, 24.5701, 19.3049, 18.8447, 30.3222]], grad_fn=<TBackward0>)


# 4. Prediction

In [14]:
probs = logits_per_image.softmax(dim=1)

# probs에서 인덱스 최대값을 찾아서 item()을 통해 숫자형식으로 출력
print(candidates[torch.argmax(probs).item()])

two cats lying on the cushion
